In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv
/kaggle/input/sentiment-analysis-on-moroccan-arabic-dialect/test_stage1.csv
/kaggle/input/sentiment-analysis-on-moroccan-arabic-dialect/train.csv


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import NuSVC
from sklearn.metrics import confusion_matrix ,recall_score
import seaborn as sns

from sklearn import metrics

In [3]:
df = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/train.csv")
#df.columns = ['comment','label']
test = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/test_stage1.csv")

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       240 non-null    int64 
 1   comment  240 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.9+ KB


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('arabic')
import re
#
import string
translator = str.maketrans('', '', string.punctuation)
#Stemming:
#Arabic Light Stemming

## 2 bibliothéques pour stemming utiliser l'une des deux 
## pour TF IDF utiliser ISRIStemmer 
## pour word2vec et  FastText utiliser Farasa

In [7]:
from nltk.stem.isri import ISRIStemmer
Stemmer=ISRIStemmer()

In [8]:
!pip install farasapy

In [9]:
from farasa.stemmer import FarasaStemmer
Stemmer = FarasaStemmer('arabic')

/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


100%|██████████| 241M/241M [00:15<00:00, 15.2MiB/s]


[2022-01-21 22:28:51,194 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [10]:
def removeStopWords(text):
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
    text = ' '.join([i for i in filtered_sentence])
    return text

def NormalizeArabic(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return text

def arabic_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', text)
    return text

def removeNumbers(text):
    """ Removes integers """
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

def stemming(text):
    stemmed_words = []
    word_tokens = word_tokenize(text) 
    for w in word_tokens:
        stemmed_words.append(Stemmer.stem(w))
    stemmed_words = " ".join(stemmed_words)
    return stemmed_words

def remove_english_characters(text):
        return re.sub(r'[a-zA-Z]+', '', text)

In [11]:
for index, row in df.iterrows():
    row['comment'] = NormalizeArabic(row['comment'])
    row['comment'] = removeStopWords(row['comment'])
    row['comment'] = arabic_diacritics(row['comment'])
    row['comment'] = removeNumbers(row['comment'])
    row['comment'] = row['comment'].translate(translator)
    row['comment'] = stemming(row['comment'])
    new_df = pd.DataFrame({'comment': [row['comment']]}, index=[index])
    df.update(new_df)

In [12]:
#Arabic Pre-Pre-Processing
for index, row in test.iterrows():
    row['comment'] = NormalizeArabic(row['comment'])
    row['comment'] = removeStopWords(row['comment'])
    row['comment'] = arabic_diacritics(row['comment'])
    row['comment'] = removeNumbers(row['comment'])
    row['comment'] = row['comment'].translate(translator)
    row['comment'] = stemming(row['comment'])
    new_df = pd.DataFrame({'comment': [row['comment']]}, index=[index])
    test.update(new_df)

# **** TF IDF ****

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, encoding='latin-1', ngram_range=(0,1), stop_words= stop_words)
features = tfidf.fit_transform(df['comment']).toarray()
labels = df['label']
features.shape

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(1920, 9236)

In [14]:
#TF-IDF Features Extraction
features_unseen = tfidf.transform(test['comment']).toarray()
features_unseen.shape

(240, 9236)

# CNB

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix ,recall_score
from sklearn.metrics import f1_score,classification_report
def cnb(c):
    X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.15, random_state=14)#32 #14
    model_cnb = ComplementNB(alpha=1,norm=False)
    scores = cross_val_score(model_cnb, features, labels, cv=5)
    print(scores)
    model_cnb.fit(X_train, y_train)
    predict=model_cnb.predict(X_test)
    print(f1_score(y_test,predict, average='weighted'))
    print(metrics.classification_report(y_test,predict))
for i in range(1):
    cnb(i)

[0.76041667 0.78645833 0.734375   0.80208333 0.76822917]
0.7527348748128146
              precision    recall  f1-score   support

           0       0.74      0.80      0.77       147
           1       0.77      0.70      0.74       141

    accuracy                           0.75       288
   macro avg       0.76      0.75      0.75       288
weighted avg       0.76      0.75      0.75       288



# **LogisticRegression**

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix ,recall_score
def reg(c):
    X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.12, random_state=3)#39#32 #14
    logreg = LogisticRegression(C=4,fit_intercept=True)
    logreg.fit(X_train, y_train)
    predict= logreg.predict(X_test)
    y_pred_unseen= logreg.predict(features_unseen)
    from sklearn.metrics import f1_score
    print(f1_score(predict, y_test, average='weighted'))
    print(recall_score(predict,y_test,average='micro'))
    print(metrics.classification_report(predict,y_test))
    submission = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv")
    submission['label'] = y_pred_unseen
    submission = submission[['ID','label']]
    #submission.to_csv('tfidf.csv', index=False)
    
for i in range(1):
    reg(i)


0.7705627705627707
0.7705627705627706
              precision    recall  f1-score   support

           0       0.75      0.79      0.77       113
           1       0.79      0.75      0.77       118

    accuracy                           0.77       231
   macro avg       0.77      0.77      0.77       231
weighted avg       0.77      0.77      0.77       231



# SGDC

In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
from sklearn.metrics import confusion_matrix ,recall_score
from sklearn.metrics import f1_score
# Always scale the input. The most convenient way is to use a pipeline.

def svm(c):
    arr=[]
    X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.15, random_state=3)#32 #14
    model_sgd = SGDClassifier(max_iter=1000, tol=1e-4,loss='log',random_state=45,alpha=0.0001,penalty='l2')
    #model_sgd = make_pipeline(StandardScaler(),SGDClassifier(max_iter=3000, loss='log',tol=1e-4,random_state =c))
    #model_sgd = linear_model.SGDClassifier(max_iter=1000,eta0=0.01, alpha=0.0001, loss='log', random_state=2, penalty='l2', tol=1e-4, learning_rate='constant')
    #clf.fit(X_train, Y)
    model_sgd.fit(X_train, y_train)
    predict=model_sgd.predict(X_test)
    y_pred_unseen = model_sgd.predict(features_unseen)
    print(f1_score(predict,y_test, average='weighted'))
    print(recall_score(predict,y_test,average='micro'))
    submission = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv")
    submission['label'] = y_pred_unseen
    submission = submission[['ID','label']]
    #submission.to_csv('tfidf(SGDC).csv', index=False)  
for i in range(1):
    svm(i)


0.7742647275587555
0.7743055555555556


# SVM

In [18]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix ,recall_score
from sklearn.metrics import f1_score
def svm(c):
    arr=[]
    X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.15, random_state=14)#32 #14
    model_svc = SVC(kernel='poly',degree=1)
    model_svc.fit(X_train, y_train)
    y_pred_svc = model_svc.predict(X_test)
    y_pred_unseen=model_svc.predict(features_unseen)
    print(f1_score(y_test, y_pred_svc, average='weighted'))
    print(recall_score(y_test,y_pred_svc ,average='micro'))
    submission = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv")
    submission['label'] = y_pred_unseen
    submission = submission[['ID','label']]
    #submission.to_csv('tfidf(svm).csv', index=False)  
for i in range(1):
    svm(i)

0.7568975225225225
0.7569444444444444


# ****Word2vec****

In [19]:
combi = df.append(test, ignore_index=True)
df.shape 

(1920, 3)

In [20]:
from gensim.models import Word2Vec
tokenized_tweet = combi['comment'].apply(lambda x: str(x).split()) # tokenizing

model_w2v = Word2Vec(tokenized_tweet, vector_size=300, window=20,min_count=5, workers=2, epochs=50)

model_w2v.train(tokenized_tweet, total_examples= len(combi['comment']), epochs=50)

(2253607, 3323150)

In [21]:
words = list(model_w2v.wv.index_to_key)
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [22]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 300))

for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 300)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape
#tokenized_tweet[252]


(2160, 300)

In [23]:
from sklearn.model_selection import train_test_split


train_w2v = wordvec_df.iloc[:1920,:]
test_w2v = wordvec_df.iloc[1920:,:]

xtrain, xvalid, ytrain, yvalid = train_test_split(train_w2v, df['label'],test_size=0.15, random_state=12)

print(wordvec_df.shape, df['label'].shape)
#len(ytrain.index)#1457
len(yvalid.index)#365
#len(train_w2v)
xtrain_w2v = train_w2v.iloc[ytrain.index,:]
xvalid_w2v = train_w2v.iloc[yvalid.index,:]


(2160, 300) (1920,)


# SVM

In [24]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.metrics import confusion_matrix ,recall_score
def  svc(c):
    train_w2v = wordvec_df.iloc[:1920,:]
    test_w2v = wordvec_df.iloc[1920:,:]

    xtrain, xvalid, ytrain, yvalid = train_test_split(train_w2v, df['label'],test_size=0.15, random_state=4)
    clf = svm.SVC(kernel='rbf',probability=False,C=1)
    clf.fit(xtrain,ytrain)
    predict = clf.predict(xvalid)
    submission2 = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv")
    test_pred = clf.predict(test_w2v)
    print(recall_score(yvalid,predict,average='micro'))
    print(f1_score(yvalid,predict, average='weighted'))
for i in range(1):
    svc(i)

0.8159722222222222
0.8159921907548289


# SGDC

In [25]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix ,recall_score
# Always scale the input. The most convenient way is to use a pipeline.

def svm(c):
    train_w2v = wordvec_df.iloc[:1920,:]
    test_w2v = wordvec_df.iloc[1920:,:]

    xtrain, xvalid, ytrain, yvalid = train_test_split(train_w2v, df['label'],test_size=0.1, random_state=39)  
    #model_sgd = SGDClassifier(max_iter=3000, tol=1e-4,loss='huber',random_state =4)
    #model_sgd = make_pipeline(StandardScaler(),SGDClassifier(max_iter=3000, tol=1e-4,random_state=0))
    model_sgd = linear_model.SGDClassifier(max_iter=3000,eta0=0.01, alpha=0.0001, loss='log', random_state=0, penalty='l2', tol=1e-4, learning_rate='constant')
    #clf.fit(X_train, Y)
    model_sgd.fit(xtrain, ytrain)
    y_pred_unseen = model_sgd.predict(test_w2v)
    predict=model_sgd.predict(xvalid)
    print(recall_score(yvalid,predict,average='micro'))
    print(f1_score(yvalid, predict, average='weighted'))
    #print('numéro:',c,metrics.classification_report(y,y_pred_unseen))    
for i in range(1):
    svm(i)


0.6979166666666666
0.6979166666666666


# Logistique Regression

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix ,recall_score
from sklearn.metrics import f1_score
def reg(c):
    train_w2v = wordvec_df.iloc[:1920,:]
    test_w2v = wordvec_df.iloc[1920:,:]
    xtrain, xvalid, ytrain, yvalid = train_test_split(train_w2v, df['label'],test_size=0.15, random_state=2) 
    logreg = LogisticRegression(C=2,fit_intercept=True,solver='liblinear')
    logreg.fit(xtrain, ytrain)
    predict=logreg.predict(xvalid)
    y_pred_unseen= logreg.predict(test_w2v)
    print(recall_score(yvalid,predict,average='micro'))
    print(f1_score(yvalid,predict, average='weighted'))
for i in range(1):
    reg(i)


0.7291666666666666
0.7286435951909613


# ****FastText****

In [27]:
from gensim.models import FastText
combi = df.append(test, ignore_index=True)
tokenized_tweet = combi['comment'].apply(lambda x: str(x).split())
print("Training a Gensim FastText model")
from gensim.test.utils import common_texts, get_tmpfile

model = FastText(tokenized_tweet,sg=4,vector_size=300,window=20,epochs=40)#241#300
print("Training complete")

Training a Gensim FastText model
Training complete


In [28]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [29]:
fasttext_arrays = np.zeros((len(tokenized_tweet),300))

for i in range(len(tokenized_tweet)):
    fasttext_arrays[i,:] = word_vector(tokenized_tweet[i], 300)
    #fasttext_arrays[i]=word_vector(tokenized_tweet[i],300)
fasttext_df = pd.DataFrame(fasttext_arrays)
fasttext_df.shape


(2160, 300)

## aprés l'éxécution de fast text nous avons extrait le vecteur suivant 

In [30]:
#fasttext_df=pd.read_csv('../input/competition/vecteur(SGDC).csv')

# SGDC

In [31]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
def sgd(c):
    train_ft = fasttext_df.iloc[:1920,:]
    test_ft = fasttext_df.iloc[1920:,:]
    xtrain, xvalid, ytrain, yvalid = train_test_split(train_ft, df['label'], test_size=0.15,random_state=99)#99
    #model_sgd = SGDClassifier(max_iter=3000, tol=1e-4,loss='log',random_state =4)
    #model_sgd = make_pipeline(SGDClassifier(max_iter=3000,eta0=0.01,alpha=0.0001,loss='log', penalty='elasticnet',tol=1e-4,random_state=21,learning_rate='constant'))#21
    model_sgd = linear_model.SGDClassifier(max_iter=1000,eta0=0.01, alpha=0.00001, loss='log', random_state=1, penalty='elasticnet', tol=1e-4,learning_rate='constant')
    #clf.fit(X_train, Y) eta0=0.01, alpha=0.0001
    model_sgd.fit(xtrain, ytrain)
    y_pred_unseen = model_sgd.predict(test_ft)
    predict=model_sgd.predict(xvalid)
    print(recall_score(yvalid,predict,average='micro'))
    print('fscore',f1_score(yvalid, predict, average='weighted'))
    #print('numéro:',c,metrics.classification_report(y,y_pred_unseen))   
    submission = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv")
    submission['label'] = y_pred_unseen
    submission = submission[['ID','label']]
    #submission.to_csv('SGDC.csv', index=False)
for i in range(1):
    sgd(i)


0.7395833333333334
fscore 0.7386727965640313


# Regression logistique

In [32]:
from sklearn.linear_model import LogisticRegression
def reg(c):
    arr=[]
    train_ft = fasttext_df.iloc[:1920,:]
    test_ft = fasttext_df.iloc[1920:,:]
    xtrain, xvalid, ytrain, yvalid = train_test_split(train_ft, df['label'], test_size=0.15,random_state=13)#66 #76
    logreg = LogisticRegression(C=1,fit_intercept=True)
    logreg.fit(xtrain, ytrain)
    y_pred_unseen= logreg.predict(test_ft)
    predict= logreg.predict(xvalid)
    print(recall_score(yvalid,predict,average='micro'))
    print(f1_score(yvalid, predict, average='weighted'))
    submission2 = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv")
    submission2['label'] = y_pred_unseen
    submission2 = submission2[['ID','label']]
    #submission2.to_csv('regress.csv', index=False)
for i in range(1):
    reg(i)


0.75
0.7487885249079278


# SVM

In [33]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn import svm
from sklearn.metrics import confusion_matrix ,recall_score
def  svc(c):
    arr=[]
    train_ft = fasttext_df.iloc[:1920,:]
    test_ft = fasttext_df.iloc[1920:,:]
    xtrain, xvalid, ytrain, yvalid = train_test_split(train_ft, df['label'], test_size=0.15,random_state=13)#12#16
    clf = svm.SVC(kernel='sigmoid',probability=False,C=1,gamma='scale')  
    clf.fit(xtrain,ytrain)
    y_pred_unseen = clf.predict(test_ft)
    predict = clf.predict(xvalid)
    print(recall_score(yvalid,predict,average='micro'))
    print(f1_score(yvalid, predict, average='weighted'))
    submission = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv")
    submission['label'] = y_pred_unseen
    submission = submission[['ID','label']]
    #submission.to_csv('SVM.csv', index=False)
for i in range(1):
    svc(i)

0.7430555555555556
0.7409442310497398
